In [ ]:
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re


In [ ]:
pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 10.7 MB/s eta 0:00:00


In [ ]:

nltk.download('punkt_tab')  # Download the Punkt tokenizer data
nltk.download('stopwords')  # Download the stopwords data



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
from rdflib import Namespace,Graph,URIRef,RDF,Literal


In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# def preprocess_text(text):
#     text = text.lower()
#     text = re.sub(r'[^\w\s]', '', text)
#     stop_words = set(stopwords.words("english"))
#     tokens = word_tokenize(text)
#     cleaned_text = " ".join([word for word in tokens if word not in stop_words])
#     return cleaned_text

# def summarize_text(text):
#     try:
#         return summarizer(text, max_length=32, min_length=10, do_sample=False)
#     except ValueError:
#         return text




In [ ]:
# EX = Namespace("http://example.org/")
# g = Graph()

# for index, entry in df.iterrows():
#     text_id = URIRef(EX[str(index)])
#     raw_content = entry["Document"]
#     label = entry["subreddit"]

#     cleaned_content = preprocess_text(raw_content)
#     summary = summarize_text(cleaned_content)

#     g.add((text_id, RDF.type, EX.TextEntry))
#     g.add((text_id, EX.summary, Literal(summary)))
#     g.add((text_id, EX.belongsToReligion, URIRef(EX[label])))

# with open("text_corpus.rdf", "wb") as f:
#     f.write(g.serialize(format="turtle").encode("utf-8"))

# print("RDF generation complete. Saved as text_corpus.rdf")

**cleaning of data**

**USING WORD BY WORD RDF**

In [ ]:
import pandas as pd
data = pd.read_csv("Text based Religion 35000.csv")
data["subreddit"] = data["subreddit"].replace({"Muslim": "islam", "MuslimMarriage": "islam", "Hijabis": "islam"})
print(data["subreddit"].value_counts())


subreddit
islam              13000
Christianity        4000
atheism             4000
hinduism            4000
exmuslim            2000
Pets                 700
gaming               700
sports               700
personalfinance      700
travel               700
Fitness              700
politics             700
food                 700
science              600
technology           600
AskReddit            600
television           600
Name: count, dtype: int64


In [ ]:
labels_to_drop = [
    "Pets", "gaming", "sports", "personalfinance", "travel",
    "Fitness", "politics", "food", "science",
    "technology", "AskReddit", "television"
]
data = data[~data["subreddit"].isin(labels_to_drop)]
print(data["subreddit"].value_counts())


subreddit
islam           13000
Christianity     4000
atheism          4000
hinduism         4000
exmuslim         2000
Name: count, dtype: int64


In [ ]:
data["subreddit"] = data["subreddit"].replace({
    "Christianity": "non muslim",
    "atheism": "non muslim",
    "hinduism": "non muslim",
    "exmuslim": "non muslim"
})

print(data["subreddit"].value_counts())

subreddit
non muslim    14000
islam         13000
Name: count, dtype: int64


In [ ]:
data.dropna(subset=['Document', 'subreddit'], inplace=True)
data.drop_duplicates(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27000 entries, 0 to 34999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      27000 non-null  int64 
 1   Document   27000 non-null  object
 2   subreddit  27000 non-null  object
dtypes: int64(1), object(2)
memory usage: 843.8+ KB


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+|#\w+', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

data['Cleaned_Document'] = data['Document'].apply(clean_text)
data[['Document', 'Cleaned_Document']].head()

                                            Document  \
0  I was raised as a Muslim, but never got into t...   
1  \nAs salamu alaikum, I wonder what the benefit...   
2  I was at a coffee shop when I saw a carbonara ...   
3  Hello, so I am a 13 year old boy, naturally I ...   
4  We intend to get married soon but my father an...   

                                    Cleaned_Document  
0  raised muslim never got habit praying five tim...  
1  salamu alaikum wonder benefit umrah hajj peopl...  
2  coffee shop saw carbonara pasta asked pork cas...  
3  hello year old boy naturally sexual desire let...  
4  intend get married soon father live great dist...  


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDF, RDFS

def preprocess_text(text):
    words = nltk.word_tokenize(text.lower())
    stopwords = set(nltk.corpus.stopwords.words('english'))
    words = [word for word in words if word.isalpha() and word not in stopwords]
    return words[:5]



In [ ]:
def create_rdf_from_csv(df, output_file="output.rdf", format="turtle"):
    g = Graph()
    base_uri = "http://example.org/religion/"

    parent_mapping = {
        "Islam": "Islam",
        "non muslim": "non_muslim",
    }

    for parent_label in parent_mapping.values():
        parent_uri = URIRef(f"{base_uri}{parent_label}")
        g.add((parent_uri, RDF.type, RDFS.Class))
        g.add((parent_uri, RDFS.label, Literal(parent_label)))

    for _, row in df.iterrows():
        text = row["Cleaned_Document"]
        label = row["subreddit"]

        parent_label = "Islam" if label == "Islam" else "non muslim"
        parent_uri = URIRef(f"{base_uri}{parent_mapping[parent_label]}")

        class_uri = URIRef(f"{base_uri}{label.replace(' ', '_')}")
        g.add((class_uri, RDF.type, RDFS.Class))
        g.add((class_uri, RDFS.label, Literal(label)))
        g.add((class_uri, RDFS.subClassOf, parent_uri))

        words = preprocess_text(text)
        unique_words = set(words)

        for word in unique_words:
            concept_uri = URIRef(f"{base_uri}{word.replace(' ', '_')}")

            query = f"""
            SELECT ?parent WHERE {{
                <{concept_uri}> rdfs:subClassOf ?parent .
            }}
            """
            existing_parents = [str(row["parent"]) for row in g.query(query)]

            if existing_parents:
                if parent_uri not in existing_parents:
                    continue

            g.add((concept_uri, RDF.type, RDFS.Class))
            g.add((concept_uri, RDFS.label, Literal(word)))
            g.add((concept_uri, RDFS.subClassOf, class_uri))

    with open(output_file, "wb") as f:
        f.write(g.serialize(format=format).encode("utf-8"))

    print(f"RDF generation complete. Saved as {output_file}")
    return g


rdf_graph = create_rdf_from_csv(data, output_file="religion_ontology.ttl", format="turtle")

In [ ]:
from rdflib import Graph, Namespace
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

g = Graph()
g.parse("religion_ontology.ttl", format="turtle")

RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def preprocess_text(text):
    words = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    return [word for word in words if word.isalpha() and word not in stop_words]

def get_rdf_embeddings():
    rdf_labels = []
    rdf_classes = []

    query = """
    SELECT ?class ?label WHERE {
        ?class rdfs:label ?label .
    }
    """
    results = g.query(query)

    for row in results:
        rdf_labels.append(str(row["label"]))
        rdf_classes.append(str(row["class"]))

    label_embeddings = embedding_model.encode(rdf_labels)
    return rdf_classes, rdf_labels, label_embeddings

def classify_text(input_text):
    words = preprocess_text(input_text)
    word_embeddings = embedding_model.encode(words)
    rdf_classes, rdf_labels, label_embeddings = get_rdf_embeddings()

    match_counts = {"Islam": 0, "non_muslim": 0}
    for word, word_emb in zip(words, word_embeddings):
        similarities = cosine_similarity([word_emb], label_embeddings)
        max_idx = similarities.argmax()
        matched_label = rdf_labels[max_idx]
        matched_class = rdf_classes[max_idx]
        similarity_score = similarities[0][max_idx]

        print(f"Word: {word}, Matched Label: {matched_label}, Similarity: {similarity_score:.3f}")

        if similarity_score > 0.7:
            query = f"""
            SELECT ?parent WHERE {{
                <{matched_class}> rdfs:subClassOf ?parent .
            }}
            """
            results = g.query(query)
            for row in results:
                parent_uri = str(row["parent"])
                if parent_uri.endswith("islam"):
                    match_counts["Islam"] += 1
                elif parent_uri.endswith("non_muslim"):
                    match_counts["non_muslim"] += 1

    print(f"Match Counts: {match_counts}")
    return "Islam" if match_counts["Islam"] > match_counts["non_muslim"] else "non_muslim"

input_text = "Salam Ali how are you doing Insha Allah we will get good grades"
classification = classify_text(input_text)

print(f"Input Text: {input_text}")
print(f"Classification: {classification}")
